In [32]:
import cv2
import numpy as np
import colour
from visualize_model import visualize
import model_predict as mdl
import torch
import glob
from dataset import make_dataloaders
import tqdm.notebook as tqdm

In [33]:
torch.manual_seed(42)
np.random.seed(42)

In [34]:
VAL_SIZE = 0.2
TEST_SIZE = 0.2
BATCH_SIZE = 16

In [35]:
from sklearn.model_selection import train_test_split

paths = glob.glob("dataset/*.jpg") # Your path for your dataset
paths_subset = np.random.choice(paths, 5000, replace=False) # choosing 1000 images randomly

train_paths, test_paths = train_test_split(paths_subset,test_size=TEST_SIZE,shuffle=True)
train_paths, val_paths = train_test_split(train_paths,test_size=VAL_SIZE / (1- TEST_SIZE),shuffle=True)

print(len(train_paths), len(val_paths), len(test_paths))

3000 1000 1000


In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
test_dl = make_dataloaders(BATCH_SIZE,paths=test_paths, split='test')

In [38]:
def delta_e(predicted, target):
    predicted_lab = cv2.cvtColor(predicted, cv2.COLOR_RGB2Lab)
    target_lab = cv2.cvtColor(target, cv2.COLOR_RGB2Lab)
    delta_E = colour.delta_E(predicted_lab, target_lab)
    return np.mean(delta_E)

In [39]:
def psnr(predicted, target):
    predicted_lab = cv2.cvtColor(predicted, cv2.COLOR_RGB2Lab)
    target_lab = cv2.cvtColor(target, cv2.COLOR_RGB2Lab)
    return cv2.PSNR(predicted_lab, target_lab)

In [40]:
def visual_eval(model, test_dl):
    data = next(iter(test_dl))
    visualize(model, data, save=True)

In [41]:
model = mdl.MainModel(net_G=mdl.build_res_unet(n_input=1, n_output=2, size=256))
model.load_state_dict(torch.load('models/res_net_unet_gan.pt', map_location=device))

model initialized with norm initialization


<All keys matched successfully>

In [42]:
def calculate_metrics():
    for data in tqdm.tqdm(test_dl):
        preds = mdl.colorize(model, data['L'])
        print(preds.shape)
        # ... DO stuff for one batch
    # Calculate metrics 

In [ ]:
calculate_metrics()
calculate_metrics()